In [1]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
%cd '../'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent


In [2]:
# https://laurentperrinet.github.io/sciblog/posts/2020-08-09-nesting-jupyter-runs.html
    
#verb =  (__name__ == "__main__")
def has_parent():
    """
    https://stackoverflow.com/questions/48067529/ipython-run-magic-n-switch-not-working
    
    Return True if this notebook is being run by calling
    %run in another notebook, False otherwise.
    """
    try:
        __file__
        # __file__ has been defined, so this notebook is 
        # being run in a parent notebook
        return True

    except NameError:
        # __file__ has not been defined, so this notebook is 
        # not being run in a parent notebook
        return False
def do_verb():
    return not has_parent()


do_it = do_verb()
if do_it : print(' Running this notebook directly = ', do_it)    


 Running this notebook directly =  True


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
# 
# https://en.wikipedia.org/wiki/ISO_8601
import datetime
timestr = datetime.datetime.now().date().isoformat()
timestr = '20201021'
timestr = '2020-11-03'
tau = 9e-4

from HOTS.Event import conv2eve
from HOTS.Tools import SaveObject, LoadObject

ds = 10
ds = 1
NbClusteringData = 2500//ds
NbTrainingData = 5000//ds
NbTestingData = 1500//ds

In [4]:
def get_nmnist(timestr, NbTrainingData, NbTestingData, NbClusteringData, DataPath='Data/testsetnmnist.p', verbose=False):
    fname_event_nmnist = f'Records/EXP_03_NMNIST/{timestr}_hots_event_nmnist.pkl'
    # print(help(LoadNMNIST))
    try:
        dataset = LoadObject(fname_event_nmnist)
        if verbose: print('loading the events from file', fname_event_nmnist)
    except:
        from HOTS.Event import LoadNMNIST
        dataset = LoadNMNIST(NbTrainingData, NbTestingData, NbClusteringData, 
                             Path=DataPath, verbose=0)
        SaveObject(dataset, fname_event_nmnist)
        if verbose: print('saving the events to file', fname_event_nmnist)

    events_train, events_test, event_cluster, labels_train, labels_test = dataset
    return events_train, events_test, event_cluster, labels_train, labels_test


In [5]:
def get_events(timestr, NbTrainingData=NbTrainingData, NbTestingData=NbTestingData, NbClusteringData=NbClusteringData, 
               tau=tau, # -> tau=1ms, si on prend 10 ms on est à 1s pour la dernière couche et les vidéos font 0.3s en moyenne
               homeo=True, homrun = False, verbose=False):
    R = 2    
    filthr = 2
    nbkNN = 3
    algo = 'lagorce'
    decay = 'exponential'
    krnlinit = 'rdn'
    nb_cluster = [4, 8, 16]

    fname_ = f'Records/EXP_03_NMNIST/{timestr}_hots_{tau*1000}'
        
    label = '_homeo' if homeo else ''
    
    fname_model = fname_ + 'ms_' + algo + label + '.pkl'
    fname_event0_o = fname_ + '_event_out_' + algo + label + '.pkl'

    if not os.path.isfile(fname_event0_o):            
        if verbose: print('creating the events in file', fname_event0_o)
        from HOTS.Event import Event, LoadNMNIST

        events_train, events_test, event_cluster, labels_train, labels_test = get_nmnist(timestr, NbTrainingData, NbTestingData, NbClusteringData)
        
        from HOTS.Layer import ClusteringLayer
        L1 = ClusteringLayer(tau=tau,R=R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=homeo,init=krnlinit)
        L2 = ClusteringLayer(tau=10 * tau,R=2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=homeo,init=krnlinit)
        L3 = ClusteringLayer(tau=10 * 10 * tau,R=2 * 2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=homeo,init=krnlinit)

        from HOTS.Network import Network
        Net = Network([L1, L2, L3])    

        if not os.path.isfile(fname_model):
            ClusterLayer, event_output = Net.TrainCluster(
                    event=event_cluster, NbClusterList=nb_cluster, to_record=True, NbCycle=1
                )
            SaveObject(ClusterLayer, fname_model)
        else: 
            if verbose: print('loading model from file', fname_model)

            ClusterLayer, Classif0 = LoadObject(fname_model)

        if verbose: print('run the events through the network')
        
        events_train_o = Net.RunNetwork(events_train, NbClusterList=ClusterLayer, homrun=homrun)
        events_test_o = Net.RunNetwork(events_test, NbClusterList=ClusterLayer, homrun=homrun)
        SaveObject([events_train, events_test], fname_event0_o)
    else: 
        if verbose: print('loading the events from file', fname_event0_o)
        events_train_o, events_test_o = LoadObject(fname_event0_o)
        
    return events_train_o, events_test_o


### Building matrix for logistic regression

In [6]:
def gather_data(events_in, labels_in, 
                tau_cla=.150, # characteristic time of a digit 
                sample_events=200, sample_space = 1,
                verbose=False, debug=False):
    
    c_int = lambda n, d : ((n - 1) // d) + 1

    n_events = events_in.time.shape[0]

    data = np.zeros((c_int(events_in.ImageSize[0], sample_space), 
                     c_int(events_in.ImageSize[1], sample_space), 
                     len(events_in.ListPolarities))) #tmp data

    X = np.zeros((c_int(n_events, sample_events), len(data.ravel())))
    y = np.zeros((c_int(n_events, sample_events), ))

    for i_event in range(1, n_events):
        
        data *= np.exp(-(events_in.time[i_event]-events_in.time[i_event-1])/tau_cla)

        x_pos = events_in.address[i_event, 0]//sample_space
        y_pos = events_in.address[i_event, 1]//sample_space
        p = events_in.polarity[i_event]
        data[x_pos, y_pos, p] = 1.

        if i_event % sample_events == sample_events//2 :
            if debug: 
                print(f'DEBUG {i_event=} {i_event//sample_events=} ')
                print(f'DEBUG {y[i_event//sample_events]=}   ')
                print(f'DEBUG  {labels_in[i_event]=} ')
            X[i_event//sample_events, :] = data.ravel()
            y[i_event//sample_events] = labels_in[i_event]
            

    if verbose: print('Number of events: ' + str(X.shape[0])+' - Number of features: ' + str(X.shape[1]))

    return X, y


### Performing logistic regression 

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html


In [7]:
from sklearn.linear_model import LogisticRegression as LR
# from sklearn.linear_model import LogisticRegressionCV as LR
from sklearn.model_selection import train_test_split

opts_LR = dict(max_iter=5000//ds, # random_state=0, 
               n_jobs=-1, class_weight='balanced')
#opts_LR['Cs'] = 5
#opts_LR['Cs'] = 32


### Performing logistic regression on raw input

In [8]:
def tic():
    global tic
    tic = time.time()
def toc():
    print(f'Done in {time.time() - tic:.3f} s')

In [9]:
if do_it : 
    tic()
    dataset = get_nmnist(timestr, NbTrainingData, NbTestingData, NbClusteringData, verbose=True)
    events_train, events_test, events_cluster, labels_train, labels_test = dataset
    toc()
    print(events_train.ListPolarities)

loading the events from file Records/EXP_03_NMNIST/2020-11-03_hots_event_nmnist.pkl
Done in 1.667 s
[0 1]


In [10]:
if do_it : 
    opts_LR = dict(random_state=0, max_iter=500, n_jobs=10, class_weight='balanced')
    opt_gather = dict(sample_events=100, sample_space = 4, verbose=True)

    tic()
    X_train, y_train = gather_data(events_train, labels_train,  **opt_gather)
    toc()

    tic()
    lr = LR(**opts_LR).fit(X_train, y_train)
    print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f}')
    toc()

    tic()
    X_test, y_test = gather_data(events_test, labels_test,  **opt_gather)
    print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f}')
    toc()
    

TypeError: 'float' object is not callable